In [1]:
import csv
import pandas as pd
import numpy  as np
import math

target = 'VANCOMYCIN'
time_steps = 14
person_count = 0
features_count = 0
dayrun = 13

#header_pre_list = ['PERSON','DAY','Y_TEST','DAYS b4 Y_TEST eq 1','ACTmax', 'ACTmin', 'ACTstd']
header_pre_list = ['PERSON','DAY','Y_TEST','DAYSb4EVENT']

In [2]:
#attention_features_{target} 
from itertools import chain
features = pd.read_csv('./saved_csvs/attention_features_{0}.csv'.format(target), header=0, index_col=0)
print("features df:", features.count, features)

features_count = features.shape[0] 
features = features.values.tolist()
features_list = list(chain.from_iterable(features)) #flatten 2D list to 1D
#print("features list:", features.count, features_list)

#Create and write the header line
header_list = header_pre_list + features_list
print("header_list", header_list)

features df: <bound method DataFrame.count of                         0
0                     bun
1                     hdl
2                     inr
3    inspired o2 fraction
4                     ldl
..                    ...
219            tacrolimus
220             trazodone
221           vasopressin
222              warfarin
223              zolpidem

[224 rows x 1 columns]>                         0
0                     bun
1                     hdl
2                     inr
3    inspired o2 fraction
4                     ldl
..                    ...
219            tacrolimus
220             trazodone
221           vasopressin
222              warfarin
223              zolpidem

[224 rows x 1 columns]
header_list ['PERSON', 'DAY', 'Y_TEST', 'DAYSb4EVENT', 'bun', 'hdl', 'inr', 'inspired o2 fraction', 'ldl', 'peep set', 'ptt', 'rbcs', 'wbcs', 'anion gap', 'atypical lymphocytes', 'bands', 'basophils', 'blood culture', 'central venous pressure', 'chloride', 'creatinine', 'daily wei

In [3]:
#======================================================================================================
#Prepare ground truths matrix and daysb4 matrix
#Y_TEST_{target}_day_{0-13} *ground truth*
#for day in range(time_steps):


for day in range(time_steps):
    y_test = pd.read_csv('./saved_csvs/Y_TEST_{0}_day_{1}.csv'.format(target, day), header=None, skiprows=1, index_col=0)
    person_count = y_test.shape[0]
    if day == 0:
        y_test_groundtruth   = np.zeros((person_count, time_steps), dtype=int)
        y_test_daysb4event   = np.zeros((person_count, time_steps), dtype=int)
        #print("y_test_groundtruth.shape:", y_test_groundtruth.shape)
        
    #print("day:", day, " y_test_groundtruth[:,day].shape:", y_test_groundtruth[:, day].shape)
    #print("y_test[1].shape:", y_test[1].shape)
    y_test_groundtruth[:, day] = y_test[1] 


In [4]:
    
#     for person in range(10):
#         print("person:", person, y_test_groundtruth[person])

#get index of days before clinical event happened e.g.
# y_test_groundtruth[person] [1, 0, 1, 0, 0, 1, 0, 0, 0, 1,  0,  0,  0,  0 ]
# y_test_daysb4event[person] [0, 1, 0, 2, 1, 0, 3, 2, 1, 0, -1, -1, -1, -1 ]
for person in range(person_count): 
    daysb4 = -1
    for day in range(time_steps-1, -1, -1): #traverse list backwards
        if y_test_groundtruth[person][day] == 1: #day of event
            daysb4 = 0
        elif daysb4 != -1:
            daysb4 += 1
        y_test_daysb4event[person][day] = daysb4
        
    if person in (6, 10, 11, 8433):    
        print("person", person, " y_test_groundtruth:", y_test_groundtruth[person])        
        print("person", person, " y_test_daysb4event:", y_test_daysb4event[person])        


person 6  y_test_groundtruth: [0 0 0 0 0 1 1 1 1 0 0 0 0 0]
person 6  y_test_daysb4event: [ 5  4  3  2  1  0  0  0  0 -1 -1 -1 -1 -1]
person 10  y_test_groundtruth: [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
person 10  y_test_daysb4event: [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
person 11  y_test_groundtruth: [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
person 11  y_test_daysb4event: [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
person 8433  y_test_groundtruth: [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
person 8433  y_test_daysb4event: [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [5]:
f = open('./saved_csvs/activations_order_{0}.csv'.format(target), 'a', newline='') #open output file
#f = open('./saved_csvs/activations_order_{0}.csv'.format(target), 'w', newline='') #w for day 0, a for the restopen output file
#f = open('./saved_csvs/activations_order_{0}_{1}.csv'.format(target,day), 'w', newline='') #open output file
writer = csv.writer(f) #create the csv writer
writer.writerow(header_list)
#=========================================================================================================
#read activations for each day
numcol = 4 + features_count
work_day = np.empty((numcol), dtype=int)
#print("work_day shape, person_count:", work_day.shape, person_count)
#for day in range(time_steps): #julia later change this to time_steps
    #activation_{target}_day_{0-13}
activations = pd.read_csv('./saved_csvs/activations_{0}_day_{1}.csv'.format(target, dayrun), index_col=0)
if isinstance(activations, pd.DataFrame):
    activations = activations.to_numpy()

#Loop through each line, remember the original order of the feature activations
#then sort the activations in ascending order, then label their order in row 2.
#then sort by row 1 back to original order
#then save row 2 as the actual order of the activation magnitude
reverse_ixorder = np.arange(start=(features_count), stop=0, step=-1)
print("features_count, activations[1].shape:", features_count, activations[1].shape)
for person in range(person_count):
    work_day[0] = person  #save person index
    work_day[1] = dayrun
    work_day[2] = y_test_groundtruth[person, dayrun] #ground truth
    work_day[3] = y_test_daysb4event[person, dayrun] #days before event

#         print("activations[person]:", activations[person])
    sort_act = np.zeros((3, features_count), dtype=float)
    sort_act[0] = np.arange(start=0, stop=(features_count), step=1)
    sort_act[1] = activations[person]
    #if person == 11:
    #    print("1: sort_act before sort:", sort_act.shape, sort_act)
    sort_act = sort_act[:, sort_act[1,:].argsort()]
    #if person == 11:
    #    print("2: sort_act after sort:", sort_act.shape, sort_act)
    #nans will sort highest so mark these as -1
    orderix = 1.0
    for j in range(features_count):
        jx = features_count - 1 - j
        if math.isnan(sort_act[1][jx]):
            sort_act[2][jx] = -1
        else:
            sort_act[2][jx] = orderix
            orderix += 1.0

    sort_act = sort_act[:, sort_act[0,:].argsort()]
    #if person == 11:
    #    print("3: sort_act after re-sort:", sort_act.shape, sort_act)
    for t in range(features_count):
        work_day[(4+t)] = int(sort_act[2, t]) #save order of activations

   
    writer.writerow(work_day)        
    
f.close() #close the output csv    "


features_count, activations[1].shape: 224 (224,)
